In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import arviz as az
import stan

from sklearn.linear_model import BayesianRidge

import nest_asyncio
nest_asyncio.apply()

# Bayesian Insurance